# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

Recall Problem 1 from hw3t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    
    # step1: reduction graph construction
    G_copy = graph.copy()
    G_copy.add_node('Source', demand = 0)
    G_copy.add_node('Sink', demand = 0)
    total_demand = 0
    total_supply = 0
    for n in G_copy.nodes:
        demand = G_copy.nodes[n]['demand']
        if demand < 0:
            G_copy.add_edge('Source',n)
            G_copy.edges['Source',n]['capacity'] = -demand
            total_supply += (-demand)
        if demand > 0:
            G_copy.add_edge(n,'Sink')
            G_copy.edges[n,'Sink']['capacity'] = demand
            total_demand += demand
            
                
    # step2: calculate maxflow
    res_dic = {}
    flow_value, flow_dict = nx.maximum_flow(G_copy, "Source", "Sink")
    
    flow_dict.pop('Source',None)
    flow_dict.pop('Sink', None)
    
    for k in flow_dict.keys():
        flow_dict[k].pop('Source',None)
        flow_dict[k].pop('Sink',None)
    
    if flow_value == total_demand and total_supply == total_demand:
        return flow_dict
    else:
        raise nx.NetworkXUnfeasible('No flow satisfying the demands')

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    div = {}
    for s1, s2_list in flow.items():
        for s2, v in s2_list.items():
            if s1 not in div.keys():
                div[s1] = -v
            else:
                div[s1] -= v
                
            if s2 not in div.keys():
                div[s2] = v
            else:
                div[s2] += v
    return div

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)